In [1]:
# load data
import numpy as np
import pickle

# pytorch
import torch.optim as optim
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import time

batch_size = 128
num_classes = 10
epochs = 5

# The data, shuffled and split between train and test sets:
(x_train, y_train) = pickle.load(open('../data/big_cifar_data.pkl', 'rb'))
y_train = y_train.flatten()
print('x_train shape:', x_train.shape)
print(y_train[:5])

y_train = y_train.astype('int64')  # for long tensor
x_train = x_train.astype('float32')
x_train = x_train / 127.5
x_train = x_train - 1
# change shape for pytorch
x_train = np.transpose(x_train, [0, 3, 2, 1])
print('new x_train shape:', x_train.shape)

x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)

test_cnt = int(len(x_train) * 0.2)
x_test = x_train[:test_cnt]
y_test = y_train[:test_cnt]
x_train = x_train[test_cnt:]
y_train = y_train[test_cnt:]

x_train shape: (20000, 32, 32, 3)
[6 9 9 4 1]
new x_train shape: (20000, 3, 32, 32)


In [2]:
print(x_train[0])
print(y_train[:5])


(0 ,.,.) = 
 -0.3255 -0.3333 -0.3098  ...  -0.4353 -0.3882 -0.3569
 -0.3882 -0.4745 -0.4353  ...  -0.4431 -0.3961 -0.3647
 -0.5608 -0.7255 -0.6627  ...  -0.4431 -0.4275 -0.3882
           ...             ⋱             ...          
 -0.4824 -0.4588 -0.4431  ...  -0.4667 -0.4275 -0.4118
 -0.5059 -0.4902 -0.4510  ...  -0.4039 -0.4118 -0.4196
 -0.5059 -0.4902 -0.4588  ...  -0.4118 -0.4118 -0.4118

(1 ,.,.) = 
  0.0824  0.1059  0.1059  ...   0.0118  0.0510  0.0667
 -0.0902 -0.2392 -0.1686  ...   0.0039  0.0275  0.0510
 -0.4039 -0.7490 -0.6706  ...  -0.0039  0.0039  0.0275
           ...             ⋱             ...          
 -0.0980 -0.0745 -0.0588  ...  -0.0510 -0.0353 -0.0353
 -0.1294 -0.1059 -0.0667  ...  -0.0588 -0.0431 -0.0353
 -0.1216 -0.1059 -0.0745  ...  -0.0510 -0.0353 -0.0353

(2 ,.,.) = 
  0.4039  0.3882  0.4510  ...   0.3333  0.3961  0.4275
  0.1529 -0.0275  0.0824  ...   0.3176  0.3804  0.4118
 -0.2549 -0.6784 -0.6157  ...   0.3176  0.3490  0.3882
           ...            

In [3]:
class CNet(nn.Module):
    def __init__(self):
        super(CNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.fc1 = nn.Linear(2304, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = F.dropout(out, p=0.25)

        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = F.max_pool2d(out, 2)
        out = F.dropout(out, p=0.25)

        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out


train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True, num_workers=4)
net = CNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(net.parameters(),lr=0.001)

x_test = Variable(x_test, volatile=True)
y_test = Variable(y_test)

In [4]:
# train
for i in range(epochs):
    print("epoch", i + 1)
    net.train()
    net.float()
    train_loss = 0
    correct = 0
    total = 0
    start_t = time.time()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # cal test loss
    net.eval()
    test_out = net(x_test)
    test_loss = criterion(test_out, y_test).data[0]

    end_t = time.time()
    time_cost = end_t - start_t
    time_cost = round(time_cost, 4)

    train_acc = round(1.0 * correct / total, 4)
    train_loss = train_loss / (batch_idx + 1)

    print("time cost", time_cost, "train loss", round(train_loss, 4), "train acc", train_acc)
    print("test loss", round(test_loss, 4))
    print("------------------------")

epoch 1
time cost 90.5453 train loss 1.9968 train acc 0.2726
test loss 1.5906
------------------------
epoch 2
time cost 87.5565 train loss 1.5466 train acc 0.439
test loss 1.4391
------------------------
epoch 3
time cost 87.8907 train loss 1.3626 train acc 0.5074
test loss 1.2535
------------------------
epoch 4
time cost 94.6281 train loss 1.2059 train acc 0.5652
test loss 1.1703
------------------------
epoch 5
time cost 88.5188 train loss 1.0664 train acc 0.6184
test loss 1.1187
------------------------
